In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
from os.path import join
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import hist
import mplhep as hep
import seaborn as sns
from coffea import util
from matplotlib import pyplot as plt
from cycler import cycler
from hist import Hist
from hist.intervals import ratio_uncertainty

sys.path.append("../")
from azh_analysis.utils.plotting import (
    get_category_labels, get_color_list, get_category_labels
)
from azh_analysis.utils.parameters import (
    get_lumis, get_categories, 
)

In [ ]:
save_plots = False

year = "2017"
lumis = get_lumis(as_picobarns=False)
source = "signal"

signal = util.load(f"../output/{source}_UL_{year}_all_OS.coffea")
MC = util.load(f"../output/MC_UL_{year}_all_OS.coffea")
cat_labels = get_category_labels()
colors = get_color_list()

outdir = f"../plots/signal/{year}"

print(signal)

In [ ]:
var = "m4l_fine"
mass_label = r"$m_{ll\tau\tau}$ [GeV]"
logscale = False
outfile = join(outdir, "signal.pdf")
mass = None


fig, axs = plt.subplots(
    nrows=1,
    ncols=2,
    figsize=(2*4.5, 4.5),
    dpi=250,
    sharex="col",
    sharey=True
)

labels = {"cons": "Constrained"}
gga = {
    int(k.split("TauM")[-1].split("_")[0]): (
        v[::sum, ::sum, ::sum, "nom", "cons", :] #+
        #v[::sum, "mm" + cat, ::sum, "nom", mass_type, :]
    )
    for k, v in signal[var].items() 
    if ("GluGluToA" in k) and ((f"M{str(mass)}" in k) or (mass is None))
}
gga = {k: gga[k] for k in np.sort(list(gga.keys()))}
gga = {
    (rf"$m_A={k}$ GeV"): v 
     for k, v in gga.items()
}
bba = {
    int(k.split("TauM")[-1].split("_")[0]): (
        v[::sum, ::sum, ::sum, "nom", "cons", :] #+
        #v[::sum, "mm" + cat, ::sum, "nom", mass_type, :]
    ) #* (0.001/0.03 if (("M225" in k) and (year=="2018")) else 1)
    for k, v in signal[var].items() 
    if ("BBA" in k) and ((f"M{str(mass)}" in k) or (mass is None))
}
bba = {k: bba[k] for k in np.sort(list(bba.keys()))}
bba = {
    (rf"$m_A={k}$ GeV"): v 
     for k, v in bba.items()
}

for j, (name, data) in enumerate({"ggA": gga, "bbA": bba}.items()):
    stack = hist.Stack.from_dict(data)
    stack.plot(
    ax=axs[j], stack=False, histtype="step", density=False
)

    axs[j].set_xlabel("")
    axs[0].set_ylabel("Density Distribution")
    hep.cms.label(data=False, year=year, ax=axs[j])
    axs[j].set_xlabel(f"{labels['cons']} " + mass_label)
    axs[j].set_xlim([200,2200])
    
axs[-1].legend(loc="center right", prop={"size": 7}, frameon=True, bbox_to_anchor=(1.4,0.5))
#axs[-1].get_legend().set_title(f"{name}")

plt.tight_layout(),
if outfile is not None:
    plt.savefig(outfile, format="pdf", dpi=800)
plt.show()

In [ ]:
var = "m4l_fine"
mass_label = r"$m_{ll\tau\tau}$ [GeV]"
logscale = False
outfile = join(outdir, "analysis-variable.pdf")
mass = 400

fig, axs = plt.subplots(
    nrows=1,
    ncols=1,
    figsize=(4,4),
    dpi=800,
    sharex="col",
    sharey=False
)

labels = {"raw": "Visible", "corr": "Corrected", "cons": "Constrained"}
for i, mass_type in enumerate(['raw', "corr", "cons"]):
    # group, cat, btags, syst, mass_type, :
    gga = {
        k: (
            v[::sum, ::sum, ::sum, "nom", mass_type, :] +
            v[::sum, ::sum, ::sum, "nom", mass_type, :]
        )
        for k, v in signal[var].items() 
        if ("GluGluToA" in k) and ((f"M{str(mass)}" in k) or (mass is None))
    }
    gga = {
        (labels[mass_type]): v 
         for k, v in gga.items()
    }
    
    stack = hist.Stack.from_dict(gga)
    stack.plot(
        ax=axs, stack=False, histtype="step", density=False
    )
    
axs.set_xlabel("")
axs.set_ylabel("")
axs.set_xlim([100, 600])
axs.legend(loc="upper left", prop={"size": 8}, frameon=True)
hep.cms.label(data=False, year=year, ax=axs)
axs.set_xlabel(r"$m_{ll\tau\tau}$ [GeV]")

plt.tight_layout(),
if outfile is not None:
    plt.savefig(outfile, format="pdf", dpi=800)
plt.show()

In [ ]:
var = "m4l_fine"
mass_label = r"$m_{ll\tau\tau}$ [GeV]"
logscale = False
outfile = None
mass = None


fig, axs = plt.subplots(
    nrows=2,
    ncols=3,
    figsize=(12.5, 2*12.5/3),
    dpi=800,
    sharex="col",
    sharey=False
)

labels = {"raw": "Raw", "corr": "Corrected", "cons": "Constrained"}
for cat, cat_label in cat_labels.items():
    for i, mass_type in enumerate(['raw', "corr", "cons"]):
        # group, cat, btags, syst, mass_type, :
        gga = {
            k: (
                v[::sum, "ee" + cat, 0, "nom", mass_type, :] +
                v[::sum, "mm" + cat, 0, "nom", mass_type, :]
            )
            for k, v in signal[var].items() 
            if ("GluGluToA" in k) and ((f"M{str(mass)}" in k) or (mass is None))
        }
        gga = {
            (rf"$m_A={k.split('TauTauM')[-1]}$ GeV"): v 
             for k, v in gga.items()
        }
        bba = {
            k: (
                v[::sum, "ee" + cat, 1, "nom", mass_type, :] +
                v[::sum, "mm" + cat, 1, "nom", mass_type, :]
            ) * (0.001/0.03 if "M225" in k else 1)
            for k, v in signal[var].items() 
            if ("BBA" in k) and ((f"M{str(mass)}" in k) or (mass is None))
        }
        bba = {
            (rf"$m_A={k.split('TauTauM')[-1]}$ GeV"): v 
             for k, v in bba.items()
        }
            
        for j, data in enumerate([gga, bba]):
            stack = hist.Stack.from_dict(data)
            stack.plot(
            ax=axs[j, i], stack=False, histtype="step", density=False
        )

            axs[j, i].set_xlabel("")
            axs[j, i].set_ylabel("")
            #axs[j, i].legend(loc="best", prop={"size": 16}, frameon=True)
            #axs[j, i].get_legend().set_title(f"{cat_label}")
            hep.cms.label("Preliminary", data=False, lumi=lumis[year], year=year, ax=axs[j, i])
            axs[j, i].set_xlabel(f"{labels[mass_type]} " + mass_label)

        
    plt.tight_layout(),
    if outfile is not None:
        plt.savefig(outfile, format="pdf", dpi=800)
    plt.show()

In [ ]:
var = "m4l"
mass_label = r"$m_{ll\tau\tau}$ [GeV]"

fig, axs = plt.subplots(
    nrows=1,
    ncols=2,
    figsize=(9, 4.5),
    dpi=500,
    sharex="col",
    sharey=False
)

labels = {"raw": "Raw", "corr": "Corrected", "cons": "Constrained"}
for cat, cat_label in cat_labels.items():
    for _, mass_type in enumerate(["cons"]):
        # group, cat, btags, syst, mass_type, :
        gga = {
            k: (
                v[::sum, "ee" + cat, 0, "nom", mass_type, :] +
                v[::sum, "mm" + cat, 0, "nom", mass_type, :]
            )
            for k, v in signal[var].items() 
            if ("GluGluToA" in k) and ((f"M{str(mass)}" in k) or (mass is None))
        }
        gga = {
            (rf"$m_A={k.split('TauTauM')[-1]}$ GeV"): v 
             for k, v in gga.items()
        }
        bba = {
            k: (
                v[::sum, "ee" + cat, 1, "nom", mass_type, :] +
                v[::sum, "mm" + cat, 1, "nom", mass_type, :]
            )
            for k, v in signal[var].items() 
            if ("BBA" in k) and ((f"M{str(mass)}" in k) or (mass is None))
        }
        bba = {
            (rf"$m_A={k.split('TauTauM')[-1]}$ GeV"): v 
             for k, v in bba.items()
        }
            
        for i, data in enumerate([gga, bba]):
            stack = hist.Stack.from_dict(data)
            stack.plot(
            ax=axs[i], stack=False, histtype="step", density=False
        )

            axs[i].set_xlabel("")
            axs[i].set_ylabel("")
            axs[i].legend(loc="best", prop={"size": 5}, frameon=True)
            #axs[j, i].get_legend().set_title(f"{cat_label}")
            hep.cms.label("Preliminary", data=False, lumi=lumi, year=year, ax=axs[i])
            axs[i].set_xlabel(f"{labels[mass_type]} " + mass_label)

        
    plt.tight_layout(),
    if outfile is not None:
        plt.savefig(outfile, format="pdf", dpi=800)
    plt.show()

In [ ]:
var = "mtt"
for cat, cat_label in cat_labels.items():
    for bcat in [0]:
        nom_s = sum(nom[var].values())
        nom_s = nom_s[::sum, 'ee'+cat, 0, :, ::sum, 'nom', :] + nom_s[::sum, 'mm'+cat, 0, :, ::sum, 'nom', :]
        shift_s = sum(shift[var].values())
        shift_up = shift_s[::sum, 'ee'+cat, 0, :, ::sum, syst+"_up", :] + shift_s[::sum, 'mm'+cat, 0, :, ::sum, syst+"_up", :]
        shift_down = shift_s[::sum, 'ee'+calkklt, 0, :, ::sum, syst+"_down", :] + shift_s[::sum, 'mm'+cat, 0, :, ::sum, syst+"_down", :]
        plot_m4l_systematic(
            nom_s, shift_up, shift_down, 
            syst, cat_label, mass_label=r"$m_{\tau\tau}$ [GeV]",
            year=int(year), lumi = lumis[year],
            outfile = join(outdir, f"{source}_{cat}_{syst}_{var}.pdf") if plot else None
        )

In [ ]:
var = "mll"
for cat, cat_label in cat_labels.items():
    for bcat in [0]:
        # group, cat, sign, btag, syst, vals
        nom_s = sum(nom[var].values())
        nom_s = nom_s[::sum, 'ee'+cat, 0, ::sum, 'nom', :] + nom_s[::sum, 'mm'+cat, 0, ::sum, 'nom', :]
        shift_s = sum(shift[var].values())
        shift_up = shift_s[::sum, 'ee'+cat, 0, ::sum, syst+"_up", :] + shift_s[::sum, 'mm'+cat, 0, ::sum, syst+"_up", :]
        shift_down = shift_s[::sum, 'ee'+cat, 0, ::sum, syst+"_down", :] + shift_s[::sum, 'mm'+cat, 0, ::sum, syst+"_down", :]
        plot_systematic(
            nom_s, shift_up, shift_down,
            syst, cat_label, r"$m_{ll}$ [GeV]", 
            year=int(year), lumi = lumis[year],
            outfile = join(outdir, f"{source}_{cat}_{syst}_{var}.pdf") if plot else None
        )

In [ ]:
var = "pt"
for cat, cat_label in cat_labels.items():
    for bcat in [0]:
        for leg in ['3', '4']:
            nom_s = sum(nom[var].values())
            nom_s = nom_s[::sum, 'ee'+cat, 0, leg, ::sum, 'nom', :] + nom_s[::sum, 'mm'+cat, 0, leg, ::sum, 'nom', :]
            shift_s = sum(shift[var].values())
            shift_up = shift_s[::sum, 'ee'+cat, 0, leg, ::sum, syst+"_up", :] + shift_s[::sum, 'mm'+cat, 0, leg, ::sum, syst+"_up", :]
            shift_down = shift_s[::sum, 'ee'+cat, 0, leg, ::sum, syst+"_down", :] + shift_s[::sum, 'mm'+cat, 0, leg, ::sum, syst+"_down", :]
            plot_systematic(
                nom_s, shift_up, shift_down, 
                syst, cat_label, f"Leg {leg} " + r"$p_T$ [GeV]",
                year=int(year), lumi = lumis[year],
                outfile = join(outdir, f"{source}_{cat}_{syst}_{var}{leg}.pdf") if plot else None
            )

In [ ]:
var = "met"
for cat, cat_label in cat_labels.items():
    for bcat in [0]:
        nom_s = sum(nom[var].values())
        nom_s = nom_s[::sum, 'ee'+cat, 0, ::sum, 'nom', :] + nom_s[::sum, 'mm'+cat, 0, ::sum, 'nom', :]
        shift_s = sum(shift[var].values())
        shift_up = shift_s[::sum, 'ee'+cat, 0, ::sum, syst+"_up", :] + shift_s[::sum, 'mm'+cat, 0, ::sum, syst+"_up", :]
        shift_down = shift_s[::sum, 'ee'+cat, 0, ::sum, syst+"_down", :] + shift_s[::sum, 'mm'+cat, 0, ::sum, syst+"_down", :]
        plot_systematic(
            nom_s, shift_up, shift_down, 
            syst, cat_label, r"$E_T^\mathrm{miss}$ [GeV]",
            year=int(year), lumi = lumis[year],
            outfile = join(outdir, f"{source}_{cat}_{syst}_{var}.pdf")
        )

In [ ]:
import uproot
import numpy as np
import mplhep as hep
from matplotlib import pyplot as plt

hep.style.use(["CMS", "fira", "firamath"])
colors = {
    "DY": "#0A9396",
    "SM-H(125)": "#E9D8A6",
    "ZZ": "#94D2BD",
    "WZ": "#9b2226",
    "tt": "#EE9B00",
    "VVV": "#bb3e03",
}

masses = np.array([225, 250, 275, 300, 325, 350, 375, 400, 450, 500, 600, 700, 750, 800, 900, 1000, 1200, 1400, 1600, 1800, 2000])
#for mass in masses:
#    rootfile = uproot.open("higgsCombine.btag_mmtt.AsymptoticLimits.mH"+str(mass)+".root")
#    limit = rootfile["limit"].arrays()["limit"]
#    print(mass, limit)

limit_exp = abs(masses - 1100)/1000.
limit_obs = limit_exp + 0.1*(np.random.rand()-0.5)
limit_1s = 0.05*limit_exp
limit_2s = 0.1*limit_exp
x = np.arange(len(limit_exp))

fig, ax = plt.subplots(dpi=120)
ax.fill_between(x, limit_exp - limit_2s, limit_exp + limit_2s, color='#FFF59D', label="95% Expected")
ax.fill_between(x, limit_exp - limit_1s, limit_exp + limit_1s, color='#8BC34A', label="68% Expected")
ax.plot(limit_exp, "k--", label="Median Expected")
ax.plot(limit_obs, "k-", lw=2.5, label="Observed")
plt.legend(loc="best")
plt.tight_layout()
plt.show()

In [ ]:
import os
import uproot
print(os.listdir('root_for_combine'))
uproot.open("root_for_combine/MC_0btag_2018.root")["mmmt/ggHWW_tauESDown"].to_numpy()
